<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Current Todo:


Some ideas here: 


* Train against some sh1t coins and see if that helps and doesn't break accuracy for previously trained segments

* Maybe scaling volume directly isn't the right move. Maybe all of our volume should be turned into a ratio against bitcoins volume - and we scale that. That will help
the model potentially understand that the scaled volume is still much lower than bitcoins. Bitcoin volume would be 1. I don't know how that will vibe with the stock series volume. We may need to retrain.

* We can also add VWAP, or mVWAP and SDVWAP


## Next Todo:

* Add TA fields, use random forest to check which fields are the best, verify with mean square error (this could take a while)

* How to verify trades on highly volatile but illiquid predictions?

* SVM direction indicator?

* pandas_ta strategy/back testers look interesting?

#IMPORT DATASETS AND LIBRARIES


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')


Mounted at /content/drive


In [3]:
!pip install pandas_ta
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=3212f33c51639445586bf0c4485697ef98d79587066f44892d40d5edd321694a
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas_ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Library

In [111]:
import pandas as pd
import pandas_ta as ta
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta
import time
from os.path import exists
from decimal import *
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import KFold, ParameterGrid
from keras.layers import Input, LSTM, Attention, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

pd.options.display.float_format = '{:f}'.format
np.set_printoptions(formatter={'float': '{:f}'.format})

# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()
num_features = 2 #3
candle_features = 5 #6
coin_base = False
ku_coin = True
load_models = False
train_models = True
save_models = True
COINBASE_REST_API = 'https://api.pro.coinbase.com'
COINBASE_PRODUCTS = COINBASE_REST_API+'/products'
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_PRODUCTS = KUCOIN_REST_API+ "/api/v1/market/allTickers"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"

data_path = '/content/drive/My Drive/ml-trde-notebooks/data'
model_path = "/content/drive/My Drive/ml-trde-notebooks/models"

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()


def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_15d_slope(df):
  df['15Close'] = df['Close'].shift(15)
  df['15Date'] = df['Date'].shift(15)
  df['Trend'] = (df['Close'] - df['15Close']) / 15
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def build_attention_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  attention_layer = Attention()([x, x])
  outputs = keras.layers.Dense(1, activation='linear')(x)
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product, granularity='86400'):
  start_date = (datetime.today() - timedelta(seconds=delta*int(granularity))).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':granularity}
  response = connect(COINBASE_PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  df_history['time'] = [datetime.fromtimestamp(x) for x in df_history['time']]
  return df_history

def ku_coin_json_to_df(delta, product, granularity='86400'):
  granularity = int(granularity)
  start_date = (datetime.today() - timedelta(seconds=delta*granularity))
  end_date = datetime.now()

  # Please refer to the kucoin documentation on the expected parameters
  params = {'startAt':int(start_date.timestamp()), 'endAt':int(end_date.timestamp()), 'type':gran_to_string(granularity), 'symbol':product}
  response = connect(KUCOIN_CANDLES, params)
  response_text = response.text
  response_data = js.loads(response_text);
  if (response_data["code"] != "200000"):
    raise Exception("Illegal response: " + response_text)

  df_history = pd.DataFrame(response_data["data"])

  # kucoin is weird in that they don't have candles for everything. IF we don't have the requested
  # number of bars here, it throws off the whole algo. I don't want to try and project so we
  # just won't trade those instruments
  got_bars = len(df_history)
  if ( got_bars < delta-1):
    raise Exception("Requested:" + str(delta) + " bars " + " but only got:" + str(got_bars))

  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history['amount'] = [float(x) for x in df_history['amount']]
  return df_history

def gran_to_string(granularity):
  #todo implement this actually
  if granularity == 86400:
    return "1day"
  if granularity == 900:
    return "15min"
  raise Exception("Joe didn't implement a proper granularity to string. Lazy, lazy.")

def get_coin_data_frames(time, product, granularity='86400', feature_set = ["Close", "Volume"]):
  if coin_base:
    df_raw = coinbase_json_to_df(time, product, granularity)
  else:
    df_raw = ku_coin_json_to_df(time, product, granularity)

  if len(df_raw.index) == 0:
    print("No data for ", product)

  df_raw = df_raw.rename(columns={"time":"Date", "open":"Open", "high":"High", "low":"Low", "close":"Close", "volume":"Volume"})
  return df_raw

def build_profit_estimate(predicted, df_btc_history, budget = 5000):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def debug_prediction_frame(predicted, df_history, df_history_scaled):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Original"] = df_history_scaled[:,0]
  #Trend
  #df_predicted_chart["Original-Target"] = df_history_scaled[:,2]
  df_predicted_chart["Original-Target"] = df_history_scaled[:,1]
  df_predicted_chart["Target-Date"] = df_predicted_chart["Date"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted"] - df_predicted_chart["Original"]
  df_predicted_chart["Actual-Diff"] = df_predicted_chart["Original-Target"] - df_predicted_chart["Original"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["Close"] = df_history["Close"]
  df_predicted_chart["Target"] = df_history["Target"]
  df_predicted_chart["RealDiff"] = df_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_predicted_chart["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  return df_predicted_chart

def get_all_products():
  if coin_base:
    return get_all_coinbase_products()

  if ku_coin:
    return get_all_kucoin_products()

def get_all_kucoin_products():
  response = connect(KUCOIN_PRODUCTS, {})
  products = js.loads(response.text)
  df_products = pd.DataFrame(products["data"]["ticker"])
  df_products = df_products.rename(columns={"symbol":"id"})
  return df_products

def get_all_coinbase_products():
  response = connect(COINBASE_PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

def predict_trade(model, name, product, X, columns):
  predicted = models[test_name].predict(X).flatten()
  df_pred = pd.DataFrame(predicted, columns = ["Close"])
  for column in columns:
    if column != "Close":
      df_pred[column] = 0
  return [predicted.flatten()[-1], sc.inverse_transform(df_pred).flatten()[-1]]

def build_trade_model(predicted, df, product, name):
  # add predicted
  df_trade = df.tail(1)
  df_trade["Predicted"] = [predicted]

  # add the product, derive a move and percent
  df_trade["Product"] = product
  df_trade["Model Name"] = name
  df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
  df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
  df_trade["RawPercent"] = df_trade["Move"] / df_trade["Close"]
  df_trade["250Fees"] = (250 * 0.004) * 2
  df_trade["5kFees"] = (5000 * 0.004) * 2
  df_trade["10kFees"] = (10000 * 0.0025) * 2
  df_trade["250Profit"] = (250 * df_trade["RawPercent"]) - df_trade["250Fees"]
  df_trade["5kProfit"] = (5000 * df_trade["RawPercent"]) - df_trade["5kFees"]
  df_trade["10k0Profit"] = (10000 * df_trade["RawPercent"]) - df_trade["10kFees"]
  return df_trade


def convert_to_training_dataset(df, columns):
  df = sort_date(df)
  target_df = attachRVI(df)
  target_df = attachVWAPS(target_df)
  target_df = append_price_dif(target_df)
  features = target_df[columns]
  scaled_features = scale_data(features)
  return extract_training(scaled_features, len(features),len(features.columns)-1) + [features]

def extract_training(scaled_features, length, num_features):
  X = []
  y = []

  for i in range(0, length):
    X.append(scaled_features [i][0:num_features])
    y.append(scaled_features [i][num_features])
  X = np.asarray(X)
  y = np.asarray(y)
  return [scaled_features, X, y]

def train_model(model, X, y):

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))

  # Create an early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=5)

  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2,
      callbacks=[early_stopping]
  )
  return [model, history]

def get_group_bars(df):
  df = pd.DataFrame(sc.fit_transform(df[["Close", "Volume"]]), columns=["Close","Volume"])
  # Split into input sequences and target values
  n_steps = 4*4  # 4 hours of data at 15 minute intervals
  X = []
  Y = []
  for i in range(0, len(df), n_steps):
    df_group = df.iloc[i:i+n_steps]
    if len(df_group) != n_steps:
      continue
    X.append(np.array(df_group.values))
    Y.append(df_group.values[-1,0])

  # Convert the lists to NumPy arrays
  X = np.array(X)
  Y = np.array(Y)
  return [X, Y]

def getTrainingVanilla15mSet(ticker):
  file_path = data_path + "/" + ticker + "-15.csv"
  df = sort_date(pd.read_csv(file_path).rename(columns={"Datetime":"Date"}))
  df['Date'] = pd.to_datetime(df['Date'])
  return get_group_bars(df)


def build_15m_model(getTrainingSet=getTrainingVanilla15mSet):
  # the 15 min bar model
  # Build the model
  group_size = 4*4
  features = 2
  model15 = build_model(group_size, features)

  # Compile the model
  model15.compile(loss='mean_squared_error', optimizer='adam')

  # Train it
  tickers = ["SPY", "IBM", "TSLA", "CAT", "XOM", "B", "F", "AAPL", "AMZN"]

  early_stopping = EarlyStopping(monitor='val_loss', patience=5)
  for ticker in tickers:
    [X,Y] = getTrainingSet(ticker)
    model15.fit(X, Y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2,callbacks=[early_stopping])
  return model15

def fetch_and_predict_short_term(model, product):
  if coin_base:
    df_raw = coinbase_json_to_df(16, product, 900)
  else:
    df_raw = ku_coin_json_to_df(16, product, 900)
  df_raw = df_raw.rename(columns={"close":"Close", "volume": "Volume"})
  [X, Y] = get_group_bars(df_raw[["Close", "Volume"]])
  predicted = model.predict(X)
  df_pred = pd.DataFrame(predicted, columns = ["Close"])
  df_pred["Volume"] = 0
  return [predicted.flatten()[0], sc.inverse_transform(df_pred).flatten()[0]]

def attachVWAPS(df, length=30):
  vwaps = df
  vwaps.set_index(pd.DatetimeIndex(vwaps["Date"]), inplace=True)
  vwaps["VWAP"] = df.ta.vwap(length=length)
  vwaps = vwaps.dropna(subset=["VWAP"])
  vwaps['VWAPD'] = vwaps['Close'] - vwaps['VWAP']
  return vwaps

def attachRVI(df):
  vol_df = df
  vol_df["RVI"] = df.ta.rvi()
  return vol_df.dropna(subset=["RVI"])


#pull training data
all_stocks_price_df = sort_date(pd.read_csv(data_path+'/stock.csv'))
all_stocks_vol_df = sort_date(pd.read_csv(data_path+"/stock_volume.csv"))
spy_df = sort_date(pd.read_csv(data_path+'/SPY.csv'))
cat_df = sort_date(pd.read_csv(data_path+'/CAT.csv'))
f_df = sort_date(pd.read_csv(data_path+'/F.csv'))
xom_df = sort_date(pd.read_csv(data_path+'/XOM.csv'))
ibm_df = sort_date(pd.read_csv(data_path+'/IBM.csv'))
spy_15_df = sort_date(pd.read_csv(data_path+'/SPY-15.csv').rename(columns={"Datetime":"Date"}))

# Model Config and Supporting Functions

In [104]:
models = {}
all_features = ["Close", "Volume", "RVI", "VWAP", "VWAPD"]
target_column = ["Target"]
all_columns = all_features + target_column

model_config = {
    "day_bar_models": [
        "lstm_cv", 
        "lstm_att_cv", 
        "lstm_att_ohlcv", 
        "lstm_cv_rvi",
        "lstm_cv_vwap", 
        "lstm_ohlc",
        "svm_cv",
        "svm_cv_vwap"],
    "15m_bars": ["lstm_15m"],
    "column_sets": {
        "lstm_cv": ["Close", "Volume"],
        "lstm_ohlc": ["Open", "High", "Low", "Close", "Volume"],
        "lstm_att_cv": ["Close", "Volume"],
        "lstm_att_ohlcv": ["Open", "High", "Low", "Close", "Volume"],
        "lstm_cv_rvi": ["Close", "Volume", "RVI"],
        "lstm_cv_vwap": ["Close", "Volume", "VWAP", "VWAPD"],
        "svm_cv": ["Close", "Volume"],
        "svm_cv_vwap": ["Close", "Volume", "VWAP", "VWAPD"]
    },
    "build_type":{
        "lstm_att_cv":"att",
        "lstm_att_ohlcv":"att"
    },
    "load_type": {
      "svm_cv": "joblib",
      "svm_cv_vwap": "joblib"
    }
}

def get_raw_data_for_coin_list(days, coins = ["FCON-USDT", "GMT3L-USDT", "NEAR3L-USDT", "H2O-USDT", "DOGE3L-USDT", "DOGE3S-USDT"]):
  coin_dfs = {}
  for coin in coins:
    print("coin: ", coin)
    coin_dfs[coin] = get_coin_data_frames(days, coin)
    time.sleep(1)
  return coin_dfs
  
def build_model_from_config(models, name, config):
  if not name in models:
    print("newing: ", name)
    features = config["column_sets"][name]
    num_features = len(features)
    if name in config["build_type"] and config["build_type"][name] == "att":
      models[name] = build_attention_model(num_features, 1)
    if name in config["build_type"] and config["build_type"][name] == "svm":
      models[name] = SVR(kernel='rbf')
    else:
      models[name] = build_model(num_features, 1)

def train_config_model_against_df(name, config, df):
  features = config["column_sets"][name]
  num_features = len(features)
  print("Data prep for:", name)
  [scaled_features, X, y, normal_features] = convert_to_training_dataset(df, columns=features+target_column)  
  print("training:", name)
  model = models[name]
  history = model.fit(X, y)
  return [model, history]    

def build_and_stash_all_config_models():
    for name in model_config["day_bar_models"]:
      build_model_from_config(models, name, model_config)

# Load all models from disk

In [26]:
lstm_15m = keras.models.load_model(model_path + "/lstm_15m.h15")

In [ ]:
if load_models:
  for name in model_config["day_bars"]:
    print("loading:", name)
    if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
      models[name] = joblib.load(model_path + "/" + name + ".joblib") 
    else:
      models[name] = keras.models.load_model(model_path + "/" + name + ".h15")
  lstm_15 = keras.models.load_model(model_path + "/lstm_15m.h15")
  print ("models loaded")  

else:
  print ("Not loading")
  


# Train models


In [ ]:
histories = {}
if train_models:
  build_and_stash_all_config_models()
  coin_dfs = list(get_raw_data_for_coin_list(180).values())
  stock_dfs = [spy_df, cat_df, f_df, ibm_df, xom_df]
  training_dfs = coin_dfs + stock_dfs
  for df in training_dfs:
    for name in model_config["day_bar_models"]:
      history = train_config_model_against_df(name, data, df)
    if not name in histories:
      histories[name] = []
    histories[name].append(history)
  lstm_15m = build_15m_model

## Train a single model against a single dataset

In [106]:
build_and_stash_all_config_models()
#coin_dfs = get_raw_data_for_coin_list(180)

newing:  lstm_cv
newing:  lstm_att_cv
newing:  lstm_att_ohlcv
newing:  lstm_cv_rvi
newing:  lstm_cv_vwap
newing:  lstm_ohlc
newing:  svm_cv
newing:  svm_cv_vwap


In [113]:
#[model, history] = train_config_model_against_df("lstm_cv", model_config, coin_dfs["FCON-USDT"])
#models["lstm_cv"] = model
df_raw = get_coin_data_frames(180, "FCON-USDT")
test_name = "lstm_ohlc"
all_columns = model_config["column_sets"][test_name]+target_column
[scaled_features, X, y, normal_features] = convert_to_training_dataset(df, columns=all_columns)  
[p_scaled, p] = predict_trade(models[test_name], test_name, "FCON-USDT", X, all_columns)
build_trade_model(p, normal_features, "FCON-USDT", test_name)

6/6 [==============================] - 0s 14ms/step


,Open,High,Low,Close,Volume,Target,Predicted,Product,Model Name,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
Date,,,,,,,,,,,,,,,,,,
2023-01-06,0.000046,0.000052,0.000046,0.000046,2099334268.137800,0.000048,0.000043,FCON-USDT,lstm_ohlc,-0.000004,-7.741935,-0.077419,2.000000,40.000000,50.000000,-21.354839,-427.096774,-824.193548


# Save all models to disk

In [ ]:
if save_models:
  for name in model_config["day_bar_models"]:
    print("saving:", name)
    if name in model_config["load_type"] and model_config["load_type"][name] == "joblib":
      joblib.dump(models[name], model_path + "/" + name + ".joblib") 
    else:
      models[name].save(model_path + "/" + name + ".h15")
  print ("models saved")  
else:
  print ("Not saving")


# Visualize and Compare all Models

In [ ]:
#data
[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(60, "DOGE3L-USDT")

#operations
rvi_history = attachRVI(btc_history)
vwap_history = attachVWAPS(rvi_history, 30)

df_chart = pd.DataFrame();
df_chart["Date"] = rvi_history["Date"]

X = scale_data(vwap_history[["Close", "Volume", "RVI", "VWAP", "VWAPD"]])
scaled_close = X[:,[0]]
df_chart["Close"] = scaled_close

graph_columns = ["Date","Close"]

results = pd.DataFrame();
for key, model in models.items():
  
  graph_columns.append(key)
  
  # colums = todo, supply columns to each and pluck unwanted so this
  columns = model_config["column_sets"][key]
  fields = list(range(len(columns)))

  # todo recompair model orig
  x = X[:,fields]
  predicted = model.predict(x).flatten()

  mse = mean_squared_error(scaled_close, predicted)
  print("predicted mse for model: ", key, mse)
  
  [df_profit, profit] = build_profit_estimate(predicted, vwap_history)
  print("profit for model: ", key, profit)

  results = results.append([[key, mse, profit]])

  df_chart[key] = predicted

interactive_plot(df_chart[graph_columns], "wow") 
results

# Get or Train a Model

# What has a buy indicator for tomorrow?

In [ ]:
[predicted_scaled, predicted] = fetch_and_predict_short_term(lstm_15m, "BTC-USDT")
[predicted_scaled, predicted] 

                  time         open        Close         high          low  \
0  2023-01-03 10:15:00 16734.900000 16735.000000 16735.000000 16734.900000   
1  2023-01-03 10:00:00 16738.900000 16735.000000 16747.900000 16734.300000   
2  2023-01-03 09:45:00 16745.400000 16738.900000 16756.100000 16736.300000   
3  2023-01-03 09:30:00 16726.700000 16745.400000 16745.500000 16726.600000   
4  2023-01-03 09:15:00 16725.200000 16726.600000 16732.100000 16725.200000   
5  2023-01-03 09:00:00 16715.700000 16725.200000 16726.300000 16715.600000   
6  2023-01-03 08:45:00 16700.000000 16715.600000 16715.700000 16699.800000   
7  2023-01-03 08:30:00 16712.800000 16700.100000 16715.300000 16700.000000   
8  2023-01-03 08:15:00 16714.000000 16712.800000 16714.500000 16708.300000   
9  2023-01-03 08:00:00 16730.700000 16714.000000 16735.500000 16713.900000   
10 2023-01-03 07:45:00 16719.100000 16730.800000 16730.800000 16719.100000   
11 2023-01-03 07:30:00 16714.000000 16719.100000 16720.400000 16

[0.46584895, 16721.202957549693]

In [47]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]
models_in_play = ["lstm_cv"]
if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
df_estc = pd.DataFrame(); #expected short term closes
df_estc["Product"] = [];
df_estc["Est Close"] = [];
df_estc["Est Close Raw"] = [];
bars = 91
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    df_raw = get_coin_data_frames(bars, row.id)
    
    for name in models_in_play:
      print("predicting trade:", name, "for", row.id)
      columns = model_config["column_sets"][name]+target_column
      [scaled_features, X, y, normal_features] = convert_to_training_dataset(df, columns)  
      [prediction_scaled, prediction] = predict_trade(models[name], name, row.id, X, columns)
      df_trade = build_trade_model(prediction, normal_features)
      df_trades = df_trades.append(df_trade)

    try:
      print("predicting 15m:", name, "for", row.id)
      [predicted_scaled, predicted] = fetch_and_predict_short_term(lstm_15m, row.id)
      df2 = pd.DataFrame({'Product': [row.id], 'Est Close': [predicted], 'Est Close Raw': predicted_scaled})
      df_estc = df_estc.append(df2)
    except Exception as inst_15:
      print("15m Error: ", inst_15)
   
  except Exception as inst:
    #raise inst
    print("Error: ", inst)

  time.sleep(1)

df_trades.reset_index()
df_buys = df_trades[df_trades['Move'] > 0] 
df_shorts = df_trades[df_trades['Move'] < 0] 


fetching:  NKN-USDT
predicting trade: lstm_cv for NKN-USDT
3/3 [==============================] - 0s 9ms/step
predicting 15m: lstm_cv for NKN-USDT
1/1 [==============================] - 0s 33ms/step
fetching:  GEM-USDT
Error:  Requested:91 bars  but only got:89
fetching:  CUSD-USDT
predicting trade: lstm_cv for CUSD-USDT
3/3 [==============================] - 0s 9ms/step
predicting 15m: lstm_cv for CUSD-USDT
15m Error:  Requested:16 bars  but only got:5
fetching:  LTC3L-USDT
predicting trade: lstm_cv for LTC3L-USDT
3/3 [==============================] - 0s 10ms/step
predicting 15m: lstm_cv for LTC3L-USDT
15m Error:  Requested:16 bars  but only got:14


KeyboardInterrupt: ignored

In [38]:
df_buys.to_csv(data_path+"/buys-01-03-2023.csv")
df_estc.to_csv(data_path+"/15m-pred-01-03-2023.csv")
df_shorts.to_csv(data_path+"/shorts-01-03-2023.csv")
df_buys



,Close,Volume,Predicted,Product,Model Name,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit
77,0.019160,1067676.069200,0.019192,FLAME-USDT,lstm_cv,0.000032,0.169194,0.001692,2.000000,40.000000,50.000000,-1.577016,-31.540312,-33.080624
77,4.270000,5731.939700,4.282198,UNIC-USDT,lstm_cv,0.012198,0.285661,0.002857,2.000000,40.000000,50.000000,-1.285849,-25.716975,-21.433949
77,0.003077,1907883.928200,0.003094,BULL-USDT,lstm_cv,0.000017,0.552780,0.005528,2.000000,40.000000,50.000000,-0.618051,-12.361022,5.277956
77,0.015380,2408977.194900,0.015397,TONE-USDT,lstm_cv,0.000017,0.110527,0.001105,2.000000,40.000000,50.000000,-1.723681,-34.473625,-38.947250
77,0.054400,32621.035700,0.055764,LTC3S-USDT,lstm_cv,0.001364,2.507328,0.025073,2.000000,40.000000,50.000000,4.268321,85.366414,200.732827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.536000,1254.804800,0.538862,QUARTZ-USDT,lstm_cv,0.002862,0.533916,0.005339,2.000000,40.000000,50.000000,-0.665210,-13.304192,3.391616
77,0.076100,415660.996400,0.076574,MC-USDT,lstm_cv,0.000474,0.622478,0.006225,2.000000,40.000000,50.000000,-0.443804,-8.876077,12.247845
77,0.002260,3790053.441900,0.002265,KLV-USDT,lstm_cv,0.000005,0.199341,0.001993,2.000000,40.000000,50.000000,-1.501649,-30.032972,-30.065945
77,0.017450,1449792.797500,0.017542,STORE-USDT,lstm_cv,0.000092,0.528950,0.005290,2.000000,40.000000,50.000000,-0.677624,-13.552487,2.895027


In [ ]:
df_shorts

In [39]:
df_estc

,Product,Est Close,Est Close Raw
0,NKN-USDT,0.086152,0.289954
0,KNC-USDT,0.498273,0.474825
0,LYM-USDT,0.001307,0.669301
0,HAI-USDT,0.014482,0.701095
0,MITX-USDT,0.003311,0.360461
...,...,...,...
0,UPO-USDT,0.009202,0.498222
0,2CRZ-USDT,0.001469,0.106081
0,FRR-USDT,0.000950,0.478780
0,DMTR-USDT,0.002947,0.344004


In [ ]:
[df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(90, "AMPL-USDT")
df_trade = predict_trade(lstm_cv, row.id, bars, npa_scaled)
df_trade